In [1]:
import csv
import glob
import subprocess
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing

### Extract all mutations that are known to confer resistance to rifampicin based on the NEJM data

The CRyPTIC consortium paper collected a list of genotypic features that confer drug resistnace for 4 major drug across nine genes. These genotypic features were collected trhough an extensive literature survey. The list contained SNPs (synonymous and non-synonymous) and InDels. For the purpose of this study, we only considered the SNPs that were associated with susceptibility or resistance to rifampicin. There were 10 InDels that were removed. One of the records in the table from the CRyPTIC paper, indicated that any synonymous mutation in rpoB only confered susceptibility to rifampicin. Another record indicated all non-synonymous mutations between 425 and 452 confered resistance and this was left out of the variant of interest list, since this information can then be re-capitulated from the Study_novel_exonic_variants.csv table produce for each study section. There were 6 upstream coordinates that confered susceptibility and were excluded from the variant of interest. 

#### NeST result structure

In order to anaylze the 10210 samples from the CRyPTIC consortium a HPC compatible recipe was developed for NeST and the 10000 were divided in 100 separate jobs of 100 samples each and run of the Georgia tech pace HPC framework. For each of these 100 jobs 5 summary tables were created which will be merged and used in this analysis

#### Comparison of NeST variant calls with phenotypic information from the CRyPTIC study

In order to compare the results from NeST against the phenotypic information from the CRyPTIC consortium:
1. The 90 known variants (Syn and Non-syn) variants for rpoB will be extracted from Study_known_variants.csv of each of 100 job summaries, these will be merged with the phenotypic information derieved from the literature provided by the CRyPTIC consortium
2. From the Study_novel_exonic_variants.csv all Syn mutations from the rpoB generation will be annotated as "S" for conferring susceptibility.
3. From the Study_novel_exonic_variants.csv all Non-syn mutations from the rpoB gene will be annotated as "R" for conferring resistnace.

If a sample has multiple mutations conferring either susceptibility or resistance, the sample will be marked as resistant to rifampicin.

#### Estimation of precision and recall of NeST from phenotypic information

Once the resistance status w.r.t to rifampicin for each of the samples has been compiled for the NeST samples. These will be compared with the phenotypic information from the CRyPTIC consortium for each sample and precision and recall will be calculated

### Extracting all known and novel exonic rpoB mutations from NeST runs

The result files from the 101 jobs that can be downloaded from:
http://vannberg.biology.gatech.edu/data/NEJM_results.tar.gz

The next section of code extracts all known and novel mutations in rpoB that was detected from the NeST result files. 

**NOTE** : The folder structure assumes that that NEJM_results.tar.gz was extracted in the current folder

In [2]:
## To extract rpoB known and novel variants from the NEST results the following code can be used

nest_known_variant_files = 'NEJM_tables/NEJM*/Study_known_variants.csv'
nest_known_variant_file_list = glob.glob(nest_known_variant_files)
nest_novel_exonic_variant_files = 'NEJM_tables/NEJM*/Study_novel_exonic_variants.csv'
nest_novel_exonic_variant_list = glob.glob(nest_novel_exonic_variant_files)

nest_known_rpob_mutations_file = 'NeST_results/NeST_rpoB_known_variants.csv'
nest_known_rpob_mutations = open(nest_known_rpob_mutations_file, 'w')
nest_known_rpob_mutations.write('Sample,Gene,SNP\n')
nest_known_rpob_mutations_full_file = 'NeST_results/NeST_rpoB_known_full_variants.csv'
nest_known_rpob_mutations_full = open(nest_known_rpob_mutations_full_file, 'w')

nest_novel_rpob_syn_mutations_file = 'NeST_results/NeST_rpoB_novel_syn_variants.csv'
nest_novel_rpob_syn_mutations = open(nest_novel_rpob_syn_mutations_file, 'w')
nest_novel_rpob_syn_mutations.write('Sample,Gene,SNP,Phenotype\n')

nest_novel_rpob_nsyn_mutations_file = 'NeST_results/NeST_rpoB_novel_nsyn_variants.csv'
nest_novel_rpob_nsyn_mutations = open(nest_novel_rpob_nsyn_mutations_file, 'w')
nest_novel_rpob_nsyn_mutations.write('Sample,Gene,SNP,Phenotype\n')


known_rpob = pd.DataFrame()
#Get known rpob mutations
for known_files in nest_known_variant_file_list:
    known_table = open(known_files)
    for lines in known_table:
        lines = lines.strip().split(',')
        if lines[3] == 'Gene':
            nest_known_rpob_mutations_full.write('{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13},{14},{15},{16},{17},{18}\n'.format(*lines))
        if lines[3] == 'rpoB' and lines[5] != 'WT':
            nest_known_rpob_mutations_full.write('{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13},{14},{15},{16},{17},{18}\n'.format(*lines))
            nest_known_rpob_mutations.write('{0},{1},{2}\n'.format(lines[0], lines[3], lines[4]))
nest_known_rpob_mutations.close()

for novel_files in nest_novel_exonic_variant_list:
    novel_reader= open(novel_files)
    novel_table = csv.reader(novel_reader, delimiter=',', quotechar='"')
    for lines in novel_table:
        if lines[12] == 'AAPos':
            continue
        if (lines[9] != lines[11]) and (425 <= int(lines[12]) <= 452) and (lines[3] == 'rpoB'):
            nest_novel_rpob_nsyn_mutations.write('{0},{1},{2}{3}{4},R\n'.format(lines[0], lines[3], lines[9], lines[12], lines[11]))
        elif (lines[9] == lines[11]) and (lines[3] == 'rpoB'):
            nest_novel_rpob_syn_mutations.write('{0},{1},{2}{3}{4},S\n'.format(lines[0], lines[3],lines[9], lines[12], lines[11]))

nest_novel_rpob_nsyn_mutations.close()
nest_novel_rpob_syn_mutations.close()

### Standardizing phenotypic information from Cryptic consortium 

The sample list to phenotypic information has multiple SRA numbers line or sometimes it has a biosample number rather than SRA number. This needs to be standardized

Of the 10,210 samples, 1343 of the them just had BioProjects numbers and no way to associated the sample name to the SRA file. Of the remaining 8867 samples, 185 of them had multiple SRA runs associated with them, each of which were treated separately bring the total number of samples considered to 9052. From this set 463 either did not have adequete confidence in the CRyPTIC genotype preictions or did not have phenotype information and were left out, leaving us with 8586. There are 4728 samples for which NeST called a variant in rpoB

In [3]:
var_list = pd.read_excel('CRyPTIC_tables/ResultsTableCompiled.xlsx', sheet_name='SampleListCorrected', na_values='nan')
pheno_dict = {'Sample': [], 'Phenotype': [], 'CrypticPrediction': []}
samp_dict = dict()
for index, row in var_list.iterrows():
    if pd.isnull(row['SRA']) or row['SRA'] == ' ':
        continue
    elif 'SAMN' in row['SRA']:
        samtosra = subprocess.Popen('esearch -db sra -query {0} | efetch -format docsum | xtract -pattern Runs -element Run@acc'.format(row['SRA']), shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        sraout = samtosra.communicate()[0]
        sralist = sraout.decode('utf-8').strip().split()
    else:
        sralist = row['SRA'].split()        
    for sra in sralist:
        pheno_dict['Sample'].append(sra)
        samp_dict[sra] = row['SRA']
        if row['Phenotype'] == 'nan':
            pheno_dict['Phenotype'].append(np.nan)
        else:
            pheno_dict['Phenotype'].append(row['Phenotype'])
        if row['CrypticPrediction'] == 'nan':
            pheno_dict['CrypticPrediction'].append(np.nan)
        else:
            pheno_dict['CrypticPrediction'].append(row['CrypticPrediction'])
       

In [4]:
pheno_table = pd.DataFrame(pheno_dict)
pheno_table.sort_values(by='Sample', inplace=True)

In [5]:
#Number of samples for phenotypic of cryptic predictions are present
pheno_table.replace(['F', 'U'], np.nan, inplace=True)
pheno_table.dropna(inplace=True)
pheno_table.to_excel('Phenotypes.xlsx')

### Combining mutation list to associated resistance

In [6]:
var_db = pd.read_excel('CRyPTIC_tables/ResultsTableCompiled.xlsx', sheet_name="MutationList")
var_db['Gene'], var_db['SNP'] = var_db['Mutation'].str.split('_',1).str
var_db['Variant'] = var_db['Gene'] + ':' + var_db['SNP']
var_db.head()

,Mutation,RifampicinSusceptibility,Gene,SNP,Variant
0,rpoB_A-53G,S,rpoB,A-53G,rpoB:A-53G
1,rpoB_A334D,S,rpoB,A334D,rpoB:A334D
2,rpoB_A544V,S,rpoB,A544V,rpoB:A544V
3,rpoB_A69P,S,rpoB,A69P,rpoB:A69P
4,rpoB_A857T,S,rpoB,A857T,rpoB:A857T


In [7]:
known_variants = pd.read_csv(nest_known_rpob_mutations_full_file, header=0, sep=',')
#known_variants = known_variants.loc[known_variants['FinalCall'] != "WT"]
known_variants['Variant'] = known_variants['Gene'] + ':' + known_variants['SNP']

In [8]:
known_variants = known_variants.merge(var_db, left_on='Variant', right_on='Variant', how='left')
#known_variants.loc[(known_variants['Sample'] == 'ERR108473') & (known_variants['SNP_x'] == 'S430L')]
known_variants.head()
known_variants = known_variants[['Sample', 'Gene_x', 'SNP_x', 'RifampicinSusceptibility']]

In [9]:
known_variants = known_variants.rename(columns={'Gene_x': 'Gene', 'SNP_x': 'SNP'})
known_variants.head()

,Sample,Gene,SNP,RifampicinSusceptibility
0,ERR2515344,rpoB,L452P,R
1,ERR2515345,rpoB,S450L,R
2,ERR2515346,rpoB,S450L,R
3,ERR2515347,rpoB,H445Y,R
4,ERR2515348,rpoB,S450L,R


### Create table of novel non-synonymous variants

In [10]:
nsyn_variants = pd.read_csv(nest_novel_rpob_nsyn_mutations_file, header=0, sep=',')
nsyn_variants.columns = ['Sample', 'Gene', 'SNP', 'RifampicinSusceptibility']
nsyn_variants.head()

,Sample,Gene,SNP,RifampicinSusceptibility
0,ERR2515382,rpoB,S441V,R
1,ERR2517559,rpoB,L449Q,R
2,ERR2517617,rpoB,L449Q,R
3,ERR2513382,rpoB,S441A,R
4,ERR2513382,rpoB,G442E,R


### Create table of novel synonymous variants

In [11]:
syn_variants = pd.read_csv(nest_novel_rpob_syn_mutations_file, header=0, sep=',')
syn_variants.columns = ['Sample', 'Gene', 'SNP', 'RifampicinSusceptibility']
syn_variants.head()

,Sample,Gene,SNP,RifampicinSusceptibility
0,ERR2515348,rpoB,A1075A,S
1,ERR2515355,rpoB,D103D,S
2,ERR2515359,rpoB,D103D,S
3,ERR2515360,rpoB,A1075A,S
4,ERR2515361,rpoB,D103D,S


### Merge all variant tables from NeST runs

In [12]:
final_table = known_variants.copy()
final_table = final_table.append(nsyn_variants)
final_table = final_table.append(syn_variants)
final_table.sort_values(by='Sample', inplace=True)
final_table.drop_duplicates(subset=['Sample', 'Gene'], inplace=True)
final_table = final_table[['Sample', 'RifampicinSusceptibility']]
final_table.reset_index(drop=True,inplace=True)

### Compile Phenotypes, predictions and susceptibility calls

In [13]:
final_table = final_table.rename(columns={'RifampicinSusceptibility': 'NeSTPrediction'})
final_table.head()

,Sample,NeSTPrediction
0,ERR067576,R
1,ERR067577,R
2,ERR067578,R
3,ERR067581,R
4,ERR067583,R


In [14]:
pheno_table.reset_index(drop=True, inplace=True)

In [15]:
prediction_table = pheno_table.merge(final_table, left_on='Sample', right_on='Sample', how='left')
values = {'NeSTPrediction' : 'S'}
prediction_table.fillna(value=values, inplace=True)
prediction_table.dropna(inplace=True)
prediction_table.head()

,CrypticPrediction,Phenotype,Sample,NeSTPrediction
0,R,R,ERR067576,R
1,R,R,ERR067577,R
2,R,R,ERR067578,R
3,S,S,ERR067580,S
4,R,R,ERR067581,R


In [16]:
pred_table = prediction_table.replace(['R', 'S'], [1, 0])
pred_table.to_csv('Predictions.csv', index=False)
## R code to visualize the set intersections
# preds <- read.csv('Predictions.csv', header=T, sep=",")
# upset(preds, sets = c("CrypticPrediction", "Phenotype", "NeSTPrediction"), mb.ratio = c(0.55,0.45), order.by = "degree", sets.bar.color = "#56B4E9")

In [17]:
nest = prediction_table['NeSTPrediction']
truth = prediction_table['Phenotype']
target_name = ["Resistant", "Susceptible"]
print(classification_report(truth, nest, target_names=target_name))

#cryptic_table = prediction_table[['CrypticPrediction', 'Phenotype']].replace(['F','U'], np.nan).dropna()
cryptic = prediction_table['CrypticPrediction']
target_name = ["Resistant", "Susceptible"]
print(classification_report(truth, cryptic, target_names=target_name))

cryptic = prediction_table['CrypticPrediction']
target_name = ["Resistant", "Susceptible"]
print(classification_report(cryptic, nest, target_names=target_name))

#print(classification_report(cryptic, nest, target_names=target_name))

              precision    recall  f1-score   support

   Resistant       0.97      0.88      0.92      2844
 Susceptible       0.94      0.98      0.96      5743

   micro avg       0.95      0.95      0.95      8587
   macro avg       0.95      0.93      0.94      8587
weighted avg       0.95      0.95      0.95      8587

              precision    recall  f1-score   support

   Resistant       0.97      0.98      0.97      2844
 Susceptible       0.99      0.99      0.99      5743

   micro avg       0.98      0.98      0.98      8587
   macro avg       0.98      0.98      0.98      8587
weighted avg       0.98      0.98      0.98      8587

              precision    recall  f1-score   support

   Resistant       0.99      0.90      0.95      2857
 Susceptible       0.95      1.00      0.97      5730

   micro avg       0.97      0.97      0.97      8587
   macro avg       0.97      0.95      0.96      8587
weighted avg       0.97      0.97      0.96      8587



This research was supported in part through research cyberinfrastructure resources and services provided by the Partnership for an Advanced Computing Environment (PACE) at the Georgia Institute of Technology, Atlanta, Georgia, USA.